In [ ]:
import os
import pandas as pd
import json
import folium
import copy
from itertools import chain
import numpy as np

#### 1. Europe

In [ ]:
# load the data regarding the ratio of unemployement
europe_unemploy_rate_path = r'topojson/lfsa_urgan_1_Data.csv'
europe_unemploy_rate = pd.read_csv(europe_unemploy_rate_path)
europe_unemploy_rate = europe_unemploy_rate.loc[\
    (europe_unemploy_rate.TIME==2016) & (europe_unemploy_rate.SEX=="Total") & (~europe_unemploy_rate.GEO.str.contains("European|Euro")), \
    ["GEO","Value"]]
europe_unemploy_rate.loc[europe_unemploy_rate.GEO.str.contains("Germany"), "GEO"] = "Germany"
europe_unemploy_rate.loc[europe_unemploy_rate.GEO.str.contains("Former Yugoslav Republic of Macedonia"), "GEO"] = "The former Yugoslav Republic of Macedonia"
print("We have the unemployement rate the following states: ", europe_unemploy_rate.shape[0])
europe_unemploy_rate

In [ ]:
# load the europe data (geometry of the states) 
europe_topo_path = r'topojson/europe.topojson.json'
topo_data = json.load(open(europe_topo_path))
topo_states = [state["properties"]["NAME"] for state in topo_data["objects"]["europe"]["geometries"]]
print("We draw the boundaries of the following states:")
len(topo_states), topo_states

In [ ]:
# we don't have the data of all the states!
missing_states = list(set(topo_states)-set(europe_unemploy_rate.GEO))
print("We are missing the unemployement rate about these states:")
len(missing_states), missing_states

In [ ]:
def fill_invalid(state):
    if state in list(missing_states):
        return 0.7
    else:
        return 0

In [ ]:
def optimize(data):
    used_arcs = set()
    for o in data['objects']:
        for geom in data['objects'][o]['geometries']:
            if geom['type'] == 'MultiPolygon':
                it = chain.from_iterable(geom['arcs'])
            else:
                it = geom['arcs']

            for i in chain.from_iterable(it):
                used_arcs.add(i if i >= 0 else ~i)
    for i in range(len(data['arcs'])):
        if i not in used_arcs:
            data['arcs'][i] = []
    return data

In [ ]:
europe_location = [55, 15]

m = folium.Map(location=europe_location, zoom_start=3)

# fill the states with colors depending on their unemployement rate
m.choropleth(
    geo_data=topo_data, 
    topojson='objects.europe',
    data=europe_unemploy_rate,
    columns=['GEO', 'Value'],
    #threshold_scale=[0, 5, 10, 15, 20, 25],
    key_on='properties.NAME',
    fill_color='BuGn', 
    fill_opacity=0.7, 
#     line_opacity=1,
    legend_name='Percentage of unemployement (%)')

# fill with gray the states of which we have no info and also add a popup to every state to show more information 
# about the states
for state in topo_data['objects']['europe']['geometries']:
    state_name = state["properties"]["NAME"]
    unemp_rate = europe_unemploy_rate.Value[europe_unemploy_rate.GEO == state_name]
    if unemp_rate.size == 0:
        unemp_rate = "not available"
    else:
        unemp_rate = "{0:.1f}".format(unemp_rate.iloc[0])

    tdata = copy.deepcopy(topo_data)
    tdata['objects']['europe']['geometries'] = [state]
    folium.TopoJson(
        optimize(tdata),
        'objects.europe',
        name=state['id'],
        style_function=lambda geometry: {
#                 'color' : 'transparent',
#                 'fillColor': 'trasparent',
                # grey color to the state of which we don't have an
                'fillColor': "#424949", 
                'fillOpacity': fill_invalid(geometry["properties"]["NAME"]),
            }
    ).add_child(folium.Popup("<span style=\"font-weight:bold;\"> Rate: </span> " + unemp_rate)).add_to(m)

#m.save('test.html')

m

#### 2. Switzerland

In [ ]:
# Parses a string to an integer, removing invalid characters
def parseInt(numStr):
    cleaned = [x for x in numStr if x.isdigit()]
    return int("".join(cleaned))

In [ ]:
# load the data regarding the ratio of unemployement
ch_unemploy_rate_path = r'topojson/ch_unemploy_rate.csv'
ch_unemploy_rate = pd.read_csv(ch_unemploy_rate_path)
ch_unemploy_rate = ch_unemploy_rate.loc[
    ch_unemploy_rate.Cantone != "Totale",
    ["Cantone", "Tasso di disoccupazione", "Disoccupati registrati", "Persone in cerca d'impiego"]
]

# rename the columns
column_rename = {
    "Cantone":"Cantons", 
    "Tasso di disoccupazione": "Unemployement rate", 
    "Disoccupati registrati": "Unemployed", 
    "Persone in cerca d'impiego": "People looking for a job"}
ch_unemploy_rate.rename(columns=column_rename, inplace=True)

# converto objects to integers
ch_unemploy_rate["Unemployed"] = ch_unemploy_rate["Unemployed"].apply(parseInt)
ch_unemploy_rate["People looking for a job"] = ch_unemploy_rate["People looking for a job"].apply(parseInt)

# compute the active population
ch_unemploy_rate["Active population"] = (100*ch_unemploy_rate["Unemployed"]/ch_unemploy_rate["Unemployement rate"]).astype("int")

# compute the rate of people looking for a job
ch_unemploy_rate["People looking for a job (ratio)"] = 100*ch_unemploy_rate["People looking for a job"]/ch_unemploy_rate["Active population"]
ch_unemploy_rate["People looking for a job (ratio)"] = ch_unemploy_rate["People looking for a job (ratio)"].round(decimals=1)

# sort columns 
ch_unemploy_rate = ch_unemploy_rate[["Cantons", "Active population", "People looking for a job", "People looking for a job (ratio)", "Unemployed", "Unemployement rate"]]
print("We have the unemployement rate the following cantons: ", ch_unemploy_rate.shape[0])
ch_unemploy_rate

In [ ]:
# load the swiss data (geometry of the cantons) 
ch_topo_path = r'topojson/ch-cantons.topojson.json'
topo_data_ch = json.load(open(ch_topo_path, encoding="utf-8"))
topo_cantons = [state["properties"]["name"] for state in topo_data_ch["objects"]["cantons"]["geometries"]]
print("We draw the boundaries of the following cantons:")
len(topo_cantons), topo_cantons

In [ ]:
# create dictionary to map cantons between the two datasets (key is the Italian name, value is the local name)
canton_map = {}
for i in range(ch_unemploy_rate.shape[0]):
    italian_name = ch_unemploy_rate.Cantons.iloc[i]
    local_name = topo_data_ch['objects']['cantons']["geometries"][i]["properties"]["name"]
    canton_map[italian_name] = local_name
    
# rename name column in data frame
ch_unemploy_rate.replace(to_replace = canton_map, inplace=True)

ch_unemploy_rate

In [ ]:
ch_location = [46.8, 8.5]
m = folium.Map(location=ch_location, zoom_start=8)

# fill the cantons with colors depending on their unemployement rate
m.choropleth(
    geo_data=copy.deepcopy(topo_data_ch), 
    topojson='objects.cantons',
    data=ch_unemploy_rate,
    columns=['Cantons', 'Unemployement rate'],
    #threshold_scale=[0, 5, 10, 15, 20, 25],
    key_on='properties.name',
    fill_color='BuGn', 
    fill_opacity=0.7, 
#     line_opacity=1,
    legend_name='Percentage of unemployement (%)')
m

https://www.amstat.ch/v2/definition.jsp?lang=it
The unemplyement ration is computed as unemployed people / actice population where the unemployed people are the people who are looking for a job without having one. 
"Numero dei disoccupati registrati, nel giorno di riferimento (ultimo giorno del mese), diviso per il numero di persone attive, moltiplicato per 100."
Persone attive: Persone che svolgono un’attività lucrativa di almeno un’ora per settimana e le persone non occupate.
Il tasso di disoccupazione è calcolato prendendo come denominatore il numero delle persone attive

In [ ]:
m = folium.Map(location=ch_location, zoom_start=8)

# fill the cantons with colors depending on their unemployement rate
m.choropleth(
    geo_data=copy.deepcopy(topo_data_ch), 
    topojson='objects.cantons',
    data=ch_unemploy_rate,
    columns=['Cantons', 'People looking for a job (ratio)'],
    #threshold_scale=[0, 5, 10, 15, 20, 25],
    key_on='properties.name',
    fill_color='BuGn', 
    fill_opacity=0.7, 
#     line_opacity=1,
    legend_name='Percentage of unemployement (%)')
m